In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import os
from collections import Counter
from ar_wordcloud import ArabicWordCloud
from bidi.algorithm import get_display
import arabic_reshaper
from nltk.util import ngrams


import re, html, unicodedata ,string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet


from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer 
from keras import models
from keras import layers
from keras import losses
from keras import metrics
from keras import optimizers


In [ ]:
import re , emoji, Stemmer, functools, operator, string
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("allstories.csv")
df=df[["topic","story"]]

In [ ]:
s=df.sort_values(by="topic")
s

In [ ]:
grouped = s.groupby('topic')
test_data=grouped.tail(int(1000 * 0.2))

In [ ]:
test_data["topic"].value_counts()

In [ ]:
# Remove the test data from the original DataFrame to get the training data
train_data = df.drop(test_data.index)

In [ ]:
# Print the shapes of the training and test data
print("Training data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

In [ ]:
#!pip install pyarabic
#!pip install emoji
#!pip install pystemmer

In [ ]:
st =  Stemmer.Stemmer('arabic')
def data_cleaning (text):
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'^http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r"https\S+", "", text)
  text = re.sub(r'\s+', ' ', text)
  text = re.sub("(\s\d+)","",text) 
  text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
  text = re.sub("\d+", " ", text)
  #text = ar.strip_tashkeel(text)
  #text = ar.strip_tatweel(text)
  text = text.replace("#", " ");
  text = text.replace("@", " ");
  text = text.replace("_", " ");
  translator = str.maketrans('', '', string.punctuation)
  text = text.translate(translator)
  #em = text
  #em_split_emoji = emoji.get_emoji_regexp().split(em)
  #em_split_whitespace = [substr.split() for substr in em_split_emoji]
  #em_split = functools.reduce(operator.concat, em_split_whitespace)
  #text = " ".join(em_split)
  text = re.sub(r'(.)\1+', r'\1', text)
  text_stem = " ".join([st.stemWord(i) for i in text.split()])
  text = text +" "+ text_stem
  text = text.replace("آ", "ا")
  text = text.replace("إ", "ا")
  text = text.replace("أ", "ا")
  text = text.replace("ؤ", "و")
  text = text.replace("ئ", "ي")
   
  return text

In [ ]:
train_data["cleaned"] = train_data["story"].apply(lambda x:   data_cleaning(x))


In [ ]:
train_data["cleaned"].head()

In [ ]:
 #First setting the max_len , will be useful later for BERT Model
Extra_Len = 6 # an extra padding in length , found to be useful for increasing F-score
Max_Len = train_data["cleaned"].str.split().str.len().max() + Extra_Len
print(Max_Len)

In [ ]:
label_encoder = LabelEncoder()
train_data['encodedLabel'] = label_encoder.fit_transform(train_data['topic'])
train_data.head()

In [ ]:
train_labels = to_categorical(train_data['encodedLabel'])


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_WORDS = 11838
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data["story"])
train_sequences = tokenizer.texts_to_sequences(train_data["story"])
print(len(train_sequences))
print(train_sequences[:3])
test_sequences = tokenizer.texts_to_sequences(test_data["story"])
print(len(test_sequences))
print(test_sequences[:3])

In [ ]:
MAX_LEN = 30
train_padded = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=MAX_LEN)
print(train_padded.shape)
test_padded = pad_sequences(test_sequences, padding='post', truncating='post', maxlen=MAX_LEN)
print(test_padded.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.metrics import  Precision, Recall, Accuracy

EMB_SIZE = 128

model = Sequential([
           Embedding(MAX_WORDS, EMB_SIZE, mask_zero=True, input_length=MAX_LEN),
           LSTM(128),
           Dense(11, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=[ Precision(), Recall(), Accuracy()]
)

model.summary()

In [ ]:
train_padded.shape

In [ ]:
train_labels.shape

In [ ]:
history = model.fit(train_padded,
                    train_labels,
                    epochs=10,
                    #batch_size=256,
                   )